# Combining Solar Radiation Map Data

This code takes several datasets (csv files and shapefiles) and outputs a single combined dataset with rows for each zipcode in the US and month in the year. When the dataset does not contain a zipcode, lat and long are used to match to the nearest zipcode lat and long. IF a certain zipcode is missing from the data, values are imputed using its nearest neighbor.

In [12]:
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd

## Load Datasets

In [13]:
# Average monthly DNI in Watts/square meter
# Source: https://openei.org/datasets/dataset/nrel-gis-data-continental-united-states-high-resolution-direct-normal-solar-resource
solar = gpd.read_file('../Data/Raw/lower48dnihighresolution/us9805_dni.shp')

In [14]:
# Continental US zipcodes
zipcodes = pd.read_csv('../Data/Raw/uszips.csv')
zipcodes['zip'] = zipcodes['zip'].astype('string').str.zfill(width=5)
zipcodes = zipcodes[~zipcodes['state_id'].isin(['PR', 'AK', 'HI', 'VI', 'AS', 'GU', 'MP',])].reset_index()

# Convert to point geometries for geopandas
zipcodes = gpd.GeoDataFrame(zipcodes, geometry=gpd.points_from_xy(zipcodes.lng, zipcodes.lat))
zipcodes = zipcodes.set_crs("EPSG:4326")

In [15]:
zips = zipcodes[['state_name', 'zip', 'geometry', 'lat', 'lng']]

In [16]:
# Cost of electricity
power = pd.read_csv('../Data/Processed/iouzipcodes2018.csv')
power['zip'] = power['zip'].astype('string').str.zfill(width=5)
power = power[power['res_rate'] > 0]
power = power[['zip', 'res_rate']].groupby('zip').mean()

In [17]:
# Average monthly temperature
temps = pd.read_csv('../Data/Processed/2019_monthly_avg_temps_zip.csv')
temps.rename(columns={'ZIP': 'zip', 'Sept':'Sep'}, inplace=True)
temps['zip'] = temps['zip'].astype('string').str.zfill(width=5)
temps = temps[['zip', 'Jan', 'Feb', 'Mar', 'Apr', 'May',
       'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']].groupby('zip').mean()

In [18]:
# Average monthly cloudy days
# Source: https://catalogue.ceda.ac.uk/uuid/edf8febfdaad48abb2cbaf7d7e846a86
cloud = pd.read_csv('../Data/Raw/CEDA_CRU/monthly_avg_cloud.csv')
cloud = cloud.pivot(index=['lat', 'lng'], columns="month", values="pct_cloudy_days")
cloud.dropna(subset=['Jan'], inplace=True)
cloud = cloud.reset_index()

In [19]:
# Energy usage by state
energy_use = pd.read_csv('../Data/Raw/avg_energy_consumption_per_state.csv')
energy_use.rename(columns={'Million Btu': 'annual_mbtu_used', 'kWH':'annual_kwh_used'}, inplace=True)

In [69]:
# Direct solar radiation
radiation = pd.read_csv('../Data/Raw/dni_avg_zipcode.csv')
radiation.rename(columns={'GEOID10': 'zip'}, inplace=True)
radiation['zip'] = radiation['zip'].astype('string').str.zfill(width=5)

In [21]:
zipcodes

,index,zip,lat,lng,city,state_id,state_name,zcta,parent_zcta,population,density,county_fips,county_name,county_weights,county_names_all,county_fips_all,imprecise,military,timezone,geometry
0,142,01001,42.06259,-72.62589,Agawam,MA,Massachusetts,True,NaN,17312.0,581.0,25013,Hampden,"{""25013"": ""100""}",Hampden,25013,False,False,America/New_York,POINT (-72.62589 42.06259)
1,143,01002,42.37492,-72.46210,Amherst,MA,Massachusetts,True,NaN,30014.0,210.5,25015,Hampshire,"{""25015"": ""99.03"", ""25011"": ""0.97""}",Hampshire|Franklin,25015|25011,False,False,America/New_York,POINT (-72.46210 42.37492)
2,144,01003,42.39192,-72.52479,Amherst,MA,Massachusetts,True,NaN,11357.0,6164.3,25015,Hampshire,"{""25015"": ""100""}",Hampshire,25015,False,False,America/New_York,POINT (-72.52479 42.39192)
3,145,01005,42.42017,-72.10615,Barre,MA,Massachusetts,True,NaN,5128.0,44.7,25027,Worcester,"{""25027"": ""100""}",Worcester,25027,False,False,America/New_York,POINT (-72.10615 42.42017)
4,146,01007,42.27875,-72.40036,Belchertown,MA,Massachusetts,True,NaN,15005.0,110.1,25015,Hampshire,"{""25015"": ""100""}",Hampshire,25015,False,False,America/New_York,POINT (-72.40036 42.27875)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32629,32878,99363,46.06652,-118.88846,Wallula,WA,Washington,True,NaN,350.0,3.0,53071,Walla Walla,"{""53071"": ""100""}",Walla Walla,53071,False,False,America/Los_Angeles,POINT (-118.88846 46.06652)
32630,32879,99371,46.80678,-118.31679,Washtucna,WA,Washington,True,NaN,325.0,0.6,53001,Adams,"{""53001"": ""98.71"", ""53021"": ""1.29""}",Adams|Franklin,53001|53021,False,False,America/Los_Angeles,POINT (-118.31679 46.80678)
32631,32880,99401,46.08744,-117.25143,Anatone,WA,Washington,True,NaN,642.0,1.1,53003,Asotin,"{""53003"": ""100""}",Asotin,53003,False,False,America/Los_Angeles,POINT (-117.25143 46.08744)
32632,32881,99402,46.19394,-117.14736,Asotin,WA,Washington,True,NaN,1325.0,1.8,53003,Asotin,"{""53003"": ""100""}",Asotin,53003,False,False,America/Los_Angeles,POINT (-117.14736 46.19394)


In [22]:
solar.head()

,ID,GRIDCODE,LON,LAT,DNI01,DNI02,DNI03,DNI04,DNI05,DNI06,DNI07,DNI08,DNI09,DNI10,DNI11,DNI12,DNIANN,geometry
0,1701,95454965,-95.45,49.65,2016.0,3267.0,4149.0,5121.0,4743.0,4257.0,5229.0,4599.0,3465.0,2430.0,1395.0,1530.0,3518.0,"POLYGON ((-95.40000 49.60000, -95.50000 49.600..."
1,1702,95354965,-95.35,49.65,1917.0,3141.0,4014.0,5211.0,4734.0,4095.0,5310.0,4509.0,3411.0,2439.0,1377.0,1521.0,3475.0,"POLYGON ((-95.30000 49.60000, -95.40000 49.600..."
2,1703,95254965,-95.25,49.65,1800.0,2934.0,3861.0,5139.0,4824.0,4167.0,5355.0,4545.0,3474.0,2403.0,1395.0,1413.0,3445.0,"POLYGON ((-95.20000 49.60000, -95.30000 49.600..."
3,1704,95154965,-95.15,49.65,1431.0,2403.0,3384.0,5274.0,4941.0,4185.0,5256.0,4572.0,3402.0,2331.0,1179.0,1062.0,3289.0,"POLYGON ((-95.10000 49.60000, -95.20000 49.600..."
4,1705,95054965,-95.05,49.65,1305.0,2340.0,3213.0,5418.0,4977.0,4284.0,5337.0,4617.0,3447.0,2115.0,1206.0,819.0,3260.0,"POLYGON ((-95.00000 49.60000, -95.10000 49.600..."


In [23]:
solar.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

## Merge data by zipcode and impute missing

In [24]:
from shapely.ops import nearest_points
import pandas as pd

In [25]:
from sklearn.neighbors import BallTree
import numpy as np

def get_nearest(src_points, candidates, k_neighbors=1):
    """Find nearest neighbors for all source points from a set of candidate points"""

    # Create tree from the candidate points
    tree = BallTree(candidates, leaf_size=15, metric='haversine')

    # Find closest points and distances
    distances, indices = tree.query(src_points, k=k_neighbors)

    # Transpose to get distances and indices into arrays
    distances = distances.transpose()
    indices = indices.transpose()

    # Get closest indices and distances (i.e. array at index 0)
    # note: for the second closest points, you would take index 1, etc.
    closest = indices[0]
    closest_dist = distances[0]

    # Return indices and distances
    return (closest, closest_dist)


def nearest_neighbor(left_gdf, right_gdf, return_dist=False):
    """
    For each point in left_gdf, find closest point in right GeoDataFrame and return them.

    NOTICE: Assumes that the input Points are in WGS84 projection (lat/lon).
    """

    left_geom_col = left_gdf.geometry.name
    right_geom_col = right_gdf.geometry.name

    # Ensure that index in right gdf is formed of sequential numbers
    right = right_gdf.copy().reset_index(drop=True)

    # Parse coordinates from points and insert them into a numpy array as RADIANS
    left_radians = np.array(left_gdf[left_geom_col].apply(lambda geom: (geom.x * np.pi / 180, geom.y * np.pi / 180)).to_list())
    right_radians = np.array(right[right_geom_col].apply(lambda geom: (geom.x * np.pi / 180, geom.y * np.pi / 180)).to_list())

    # Find the nearest points
    # -----------------------
    # closest ==> index in right_gdf that corresponds to the closest point
    # dist ==> distance between the nearest neighbors (in meters)

    closest, dist = get_nearest(src_points=left_radians, candidates=right_radians)

    # Return points from right GeoDataFrame that are closest to points in left GeoDataFrame
    closest_points = right.loc[closest]

    # Ensure that the index corresponds the one in left_gdf
    closest_points = closest_points.reset_index(drop=True)

    # Add distance if requested
    if return_dist:
        # Convert to meters from radians
        earth_radius = 6371000  # meters
        closest_points['distance'] = dist * earth_radius

    return closest_points

In [26]:
power_zip = zips.merge(power, how="left", on="zip")

In [27]:
missing = power_zip[power_zip['res_rate'].isnull()]
filled = power_zip[~power_zip['res_rate'].isnull()]

closest_neighbors = nearest_neighbor(missing, filled, return_dist=False)
missing['res_rate'] = closest_neighbors['res_rate'].tolist()
power_zip_filled = pd.concat([pd.DataFrame(filled), pd.DataFrame(missing)])
power_zip_filled = power_zip_filled[['zip', 'res_rate']]

/home/nadev/miniconda3/envs/geopanda/lib/python3.9/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [40]:
energy_zip_filled = zips.merge(energy_use, how="left", right_on="state", left_on="state_name")
energy_zip_filled = energy_zip_filled[['zip', 'annual_kwh_used', 'annual_mbtu_used']]
# Nothing missing to fill here, we have all states

In [70]:
radiation_zip = zips.merge(radiation, how="left", on="zip")
# Nothing missing to fill here, we have all zips

In [74]:
missing = radiation_zip[radiation_zip['dni'].isnull()]
filled = radiation_zip[~radiation_zip['dni'].isnull()]

closest_neighbors = nearest_neighbor(missing, filled, return_dist=False)
missing['dni'] = closest_neighbors['dni'].tolist()
radiation_zip_filled = pd.concat([pd.DataFrame(filled), pd.DataFrame(missing)])
radiation_zip_filled = radiation_zip_filled[['zip', 'dni']]

/home/nadev/miniconda3/envs/geopanda/lib/python3.9/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [32]:
temps_zip = zips.merge(temps, how="left", on="zip")

In [33]:
missing = temps_zip[temps_zip['Jan'].isnull()].reset_index()
filled = temps_zip[~temps_zip['Jan'].isnull()].reset_index()

closest_neighbors = nearest_neighbor(missing, filled, return_dist=False)
missing[['Jan', 'Feb', 'Mar', 'Apr', 'May',
       'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']] = closest_neighbors[['Jan', 'Feb', 'Mar', 'Apr', 'May',
       'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']]
temps_zip_filled = pd.concat([pd.DataFrame(filled), pd.DataFrame(missing)]).reset_index()
temps_zip_filled = temps_zip_filled[['zip', 'Jan', 'Feb', 'Mar', 'Apr', 'May',
       'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']]

temps_zip_filled = temps_zip_filled.set_index('zip').stack().to_frame().reset_index()
temps_zip_filled.columns = ['zip', 'month', 'temp']

In [34]:
cloud_shape = gpd.GeoDataFrame(cloud, geometry=gpd.points_from_xy(cloud.lng, cloud.lat))
cloud_zip_filled = zips.copy().reset_index()

closest_neighbors = nearest_neighbor(cloud_zip_filled, cloud_shape, return_dist=False)
cloud_zip_filled[['Jan', 'Feb', 'Mar', 'Apr', 'May',
       'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']] = closest_neighbors[['Jan', 'Feb', 'Mar', 'Apr', 'May',
       'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']]
cloud_zip_filled = cloud_zip_filled[['zip', 'Jan', 'Feb', 'Mar', 'Apr', 'May',
       'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']]

cloud_zip_filled = cloud_zip_filled.set_index('zip').stack().to_frame().reset_index()
cloud_zip_filled.columns = ['zip', 'month', 'pct_cloudy_days']

In [75]:
combined = zipcodes.merge(power_zip_filled, how="left", on="zip")
combined = combined.merge(energy_zip_filled, how="left", on="zip")
combined = combined.merge(radiation_zip_filled, how="left", on="zip")
combined = combined.merge(temps_zip_filled, how="left", on="zip")
combined = combined.merge(cloud_zip_filled, how="left", on=["zip", "month"])

In [76]:
combined = combined[['zip', 'lat', 'lng', 'timezone', 'county_name',  'city', 'state_id', 'state_name',
                     'population', 'density', 'month', 'temp', 'pct_cloudy_days', 'dni', 'res_rate', 'annual_mbtu_used', 
                     'annual_kwh_used']]

In [77]:
combined

,zip,lat,lng,timezone,county_name,city,state_id,state_name,population,density,month,temp,pct_cloudy_days,dni,res_rate,annual_mbtu_used,annual_kwh_used
0,01001,42.06259,-72.62589,America/New_York,Hampden,Agawam,MA,Massachusetts,17312.0,581.0,Jan,41.7,65.30,4.524,0.159252,63.0,18463.477410
1,01001,42.06259,-72.62589,America/New_York,Hampden,Agawam,MA,Massachusetts,17312.0,581.0,Feb,30.4,69.15,4.524,0.159252,63.0,18463.477410
2,01001,42.06259,-72.62589,America/New_York,Hampden,Agawam,MA,Massachusetts,17312.0,581.0,Mar,40.0,74.10,4.524,0.159252,63.0,18463.477410
3,01001,42.06259,-72.62589,America/New_York,Hampden,Agawam,MA,Massachusetts,17312.0,581.0,Apr,59.0,70.80,4.524,0.159252,63.0,18463.477410
4,01001,42.06259,-72.62589,America/New_York,Hampden,Agawam,MA,Massachusetts,17312.0,581.0,May,59.4,70.35,4.524,0.159252,63.0,18463.477410
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
391603,99403,46.37243,-117.25273,America/Los_Angeles,Asotin,Clarkston,WA,Washington,20489.0,51.8,Aug,90.7,39.95,4.812,0.094459,63.7,18668.627159
391604,99403,46.37243,-117.25273,America/Los_Angeles,Asotin,Clarkston,WA,Washington,20489.0,51.8,Sep,91.7,52.25,4.812,0.094459,63.7,18668.627159
391605,99403,46.37243,-117.25273,America/Los_Angeles,Asotin,Clarkston,WA,Washington,20489.0,51.8,Oct,82.2,62.75,4.812,0.094459,63.7,18668.627159
391606,99403,46.37243,-117.25273,America/Los_Angeles,Asotin,Clarkston,WA,Washington,20489.0,51.8,Nov,68.8,79.40,4.812,0.094459,63.7,18668.627159


In [78]:
unmelted = combined.pivot(index=[col for col in combined.columns if col not in ['month', 'temp', 'pct_cloudy_days']], columns='month')
unmelted.columns = list(map("_".join, unmelted.columns))
unmelted = unmelted.reset_index()

In [79]:
unmelted

,zip,lat,lng,timezone,county_name,city,state_id,state_name,population,density,...,pct_cloudy_days_Dec,pct_cloudy_days_Feb,pct_cloudy_days_Jan,pct_cloudy_days_Jul,pct_cloudy_days_Jun,pct_cloudy_days_Mar,pct_cloudy_days_May,pct_cloudy_days_Nov,pct_cloudy_days_Oct,pct_cloudy_days_Sep
0,01001,42.06259,-72.62589,America/New_York,Hampden,Agawam,MA,Massachusetts,17312.0,581.0,...,71.65,69.15,65.30,64.25,67.2,74.10,70.35,74.70,67.25,63.65
1,01002,42.37492,-72.46210,America/New_York,Hampshire,Amherst,MA,Massachusetts,30014.0,210.5,...,69.15,67.90,64.30,63.85,67.0,73.90,68.80,73.60,66.05,63.90
2,01003,42.39192,-72.52479,America/New_York,Hampshire,Amherst,MA,Massachusetts,11357.0,6164.3,...,71.65,69.15,65.30,64.25,67.2,74.10,70.35,74.70,67.25,63.65
3,01005,42.42017,-72.10615,America/New_York,Worcester,Barre,MA,Massachusetts,5128.0,44.7,...,69.15,67.90,64.30,63.85,67.0,73.90,68.80,73.60,66.05,63.90
4,01007,42.27875,-72.40036,America/New_York,Hampshire,Belchertown,MA,Massachusetts,15005.0,110.1,...,69.15,67.90,64.30,63.85,67.0,73.90,68.80,73.60,66.05,63.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32629,99363,46.06652,-118.88846,America/Los_Angeles,Walla Walla,Wallula,WA,Washington,350.0,3.0,...,85.90,86.25,85.80,37.00,56.8,72.85,61.35,80.85,63.40,53.20
32630,99371,46.80678,-118.31679,America/Los_Angeles,Adams,Washtucna,WA,Washington,325.0,0.6,...,85.75,85.35,86.65,36.95,58.1,71.45,62.65,81.35,65.05,53.85
32631,99401,46.08744,-117.25143,America/Los_Angeles,Asotin,Anatone,WA,Washington,642.0,1.1,...,83.35,84.75,86.20,37.30,59.7,73.40,64.75,79.40,62.75,52.25
32632,99402,46.19394,-117.14736,America/Los_Angeles,Asotin,Asotin,WA,Washington,1325.0,1.8,...,83.35,84.75,86.20,37.30,59.7,73.40,64.75,79.40,62.75,52.25


In [80]:
unmelted.to_csv('../Data/Processed/input_data.csv', index=False)

## Run power generated code then reload

In [114]:
df = pd.read_csv('../Data/Processed/data_with_solar_output.csv')
df['zip'] = df['zip'].astype('string').str.zfill(width=5)

In [110]:
df['annual_output_kwh_20_sps'] = [data*20/1000 for data in df['annual_output_w_hrs']]
df['percent_current_needs_met'] = df['annual_output_kwh_20_sps']/df['annual_kwh_used']
df['dollars_saved'] = df['annual_output_kwh_20_sps']*df['res_rate']

In [112]:
df

,zip,lat,lng,timezone,county_name,city,state_id,state_name,population,density,...,pct_cloudy_days_Mar,pct_cloudy_days_May,pct_cloudy_days_Nov,pct_cloudy_days_Oct,pct_cloudy_days_Sep,annual_output_w_hrs,dni,annual_output_kwh_20_sps,percent_current_needs_met,dollars_saved
0,01001,42.06259,-72.62589,America/New_York,Hampden,Agawam,MA,Massachusetts,17312.0,581.0,...,74.10,70.35,74.70,67.25,63.65,152117.987231,4.524000,3042.359745,0.164777,484.503102
1,01002,42.37492,-72.46210,America/New_York,Hampshire,Amherst,MA,Massachusetts,30014.0,210.5,...,73.90,68.80,73.60,66.05,63.90,152110.789759,4.470000,3042.215795,0.164769,516.158909
2,01003,42.39192,-72.52479,America/New_York,Hampshire,Amherst,MA,Massachusetts,11357.0,6164.3,...,74.10,70.35,74.70,67.25,63.65,152001.388434,4.404000,3040.027769,0.164651,515.787676
3,01005,42.42017,-72.10615,America/New_York,Worcester,Barre,MA,Massachusetts,5128.0,44.7,...,73.90,68.80,73.60,66.05,63.90,152326.278829,4.578000,3046.525577,0.165003,516.890130
4,01007,42.27875,-72.40036,America/New_York,Hampshire,Belchertown,MA,Massachusetts,15005.0,110.1,...,73.90,68.80,73.60,66.05,63.90,152081.411157,4.488000,3041.628223,0.164738,516.059218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32629,99363,46.06652,-118.88846,America/Los_Angeles,Walla Walla,Wallula,WA,Washington,350.0,3.0,...,72.85,61.35,80.85,63.40,53.20,148115.556681,5.178000,2962.311134,0.158679,258.825663
32630,99371,46.80678,-118.31679,America/Los_Angeles,Adams,Washtucna,WA,Washington,325.0,0.6,...,71.45,62.65,81.35,65.05,53.85,146656.020351,5.170286,2933.120407,0.157115,297.842164
32631,99401,46.08744,-117.25143,America/Los_Angeles,Asotin,Anatone,WA,Washington,642.0,1.1,...,73.40,64.75,79.40,62.75,52.25,149991.771217,4.959429,2999.835424,0.160689,304.616706
32632,99402,46.19394,-117.14736,America/Los_Angeles,Asotin,Asotin,WA,Washington,1325.0,1.8,...,73.40,64.75,79.40,62.75,52.25,148591.763811,4.868000,2971.835276,0.159189,301.773445


In [113]:
df.to_csv('../Data/Processed/data_combined.csv', index=False)